# Partners in Wine

<img src="wine.png" width="400" height="500">

## Introduction

### What is the dataset?
This dataset is various attributes describing the makeup of the wine Portuguese "Vinho Verde" wine. The classification is a rating from 1-10 to decribe the quality of the wine. 


### What will we do with this dataset?

Analysis: We analyzed the dataset with K-means clustering in order to discover what attributes are essential to having a quality wine.

Classification: We used KNN, Naive Bayes, K-means clustering, and KNN-Bagging Ensemble classifiers on our dataset. We use these to predict the quality of a wine based on various attributes of our choosing.

### What classification performed the best?
Naive Bayes and K-means clustering were both are best classifiers with an accuracy around 80%. We hypothesize that these perform the best because the classification are distributed in a way that 5 and 6 are our most common label.


In [13]:
import utils

table = utils.read_table('red_wine_quality.csv')
header = table[0]
table = table[1:]

## Data Analysis

Data Analysis: This section must provide details about the dataset. You must include:
Information about the dataset itself, e.g., the attributes and attribute types, the number of instances, and the attribute being used as the label.
Relevant summary statistics about the dataset.
Data visualizations highlighting important/interesting aspects of your dataset. Visualizations may include frequency distributions, comparisons of attributes (scatterplot, multiple frequency diagrams), box and whisker plots, etc. The goal is not to include all possible diagrams, but instead to select and highlight diagrams that provide insight about the dataset itself.
Note that this section must describe the above (in paragraph form) and not just provide diagrams and statistics. Also, each figure included must have a figure caption (Figure number and textual description) that is referenced from the text (e.g., “Figure 2 shows a frequency diagram for ...”).


### About the Dataset

The dataset has approximately 1600 instances. There are 12 attributes and they are all continuous. They are floats. The attribute being used as the label is the quality attribute. It is ranked on a scale from 1-10. [View the dataset here](https://archive.ics.uci.edu/ml/datasets/wine+quality). 

The attributes are:
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides 
- free sulfur dioxide 
- total sulfur dioxide
- density 
- pH
- sulfates
- alcohol
- **quality**

### Summary Statistics and Data Visualization

Since we aren't oenologists , we don't really know much about these attributes and their relations. Since we really only care about how it relates to the quality, we will perform K-means clustering to analyze the data. We will cluster them and see what the majority classification is for that cluster. Then, we will get the average of each attribute and see if we can spot any differences between these clusters. 

However, before we do that, it is important to view the distribution of our classifications. This will give us an insight in our future classification predictions. 

#### No. of quality ratings:
As you can see in Figure 1, the majority of the dataset is classified as either a 5 or 6. There are some 3's, 7's, and 8's but no 9 or 10. This distribution is important to be aware of when we start our classifications.

In [14]:
values, counts = utils.get_frequencies(table, header.index('quality'))
plt.figure()
plt.bar(values, counts)
plt.title("Figure 1.")
plt.show()


NameError: name 'plt' is not defined

#### K-Means Clustering to help with Data Analysis 

We can use k-means clustering to determine what attributes make for a high quality wine. K-means clustering groups up instances that are closest to eachother with k groups. This image is an example:

<img src="cluster.png" width="400" height="500">

The code below is used to find the best k for our dataset. We are going to try k=3 to k=10. We will plot the objective function scores in order to find the "elbow" point, the point at which the total sum of squares drops less drastically. 

In [ ]:
import random
import math
import copy
import numpy
import matplotlib.pyplot as plt
import k_means

def k_means_clustering(k, table):
    # Select k objects in an arbitrary fashion. Use these as the initial set of k centroids.
    centroids = k_means.select_init_centroids(k, table)
    
    match = False
    while match == False: 
        
        # Compute the distances of each instance to each centroid
        distances_table = [ [] for i in range(len(table)) ]
        for centroid in centroids:
            k_means.append_distance(table, distances_table, centroid)

        # Find the biggest distance and assign instance to that centroid
        k_means.find_centroid(distances_table)

        # Recalculate the centroids by getting the mean of each cluster
        new_centroids = k_means.recalculate_centroids(table, distances_table, [])

        # Check to see if the centroids have converged
        match = k_means.compare_centroids(centroids, new_centroids)
        centroids = new_centroids

    # Now we know what instance belongs to what cluster
    # table and distances are "parallel tables" 
    score = objective_function(table, distances_table, centroids)
    return score, distances_table, centroids
    
def objective_function(table, cluster_table, centroids):
    # Combine and group by cluster 
    new_table = k_means.combine_tables(table, cluster_table)
    cluster_index = len(new_table[0])-1
    group_names, groups = utils.group_by(new_table, cluster_index)
    
    # for each cluster, compute the sum of squares
    # add these to a cluster_total of all clusters
    total_cluster_score = 0
    for i, cluster in enumerate(groups):
        distances = []
        for row in cluster:
            distance = k_means.compute_distance(row, centroids[row[cluster_index]])
            distances.append(distance)
        total_cluster_score += sum(distances)
    print(total_cluster_score)
    return total_cluster_score
        
def find_best_cluster(table, minimum, maximum):
    objective_scores = []
    x_axis = []
    for i in range(minimum, maximum+1):
        print(i)
        score, cluster_table, centroids = k_means_clustering(i, table)
        objective_scores.append(score)
        x_axis.append(i)
    
    plt.figure()
    plt.title("Figure 2.")
    plt.plot(x_axis, objective_scores)
    plt.show()

# find the best k for clustering from 3-10.
find_best_cluster(table, 3, 10)

3
20109.461284475517
4


### Best K is 6! 

Each time this runs, the values change a little bit but it is roughly the same output everytime. The elbow point appears to be around 6 or 7 everytime, as you can see in Figure 2. We will choose to use 6 clusters. 

Now that we have found the best number of clusters, we can use them to learn more information about our dataset. 
We will look at the cluster and see what we can find out by observing the averages of each attribute for each cluster. 

In [ ]:
# cluster the dataset to form 6 groups 
score, distances_table, centroids = k_means_clustering(6, table)

# create a cluster table
cluster_table = copy.deepcopy(table)
for i, row in enumerate(cluster_table):
    cluster_table[i].append(distances_table[i][-1])

# group by cluster 
group_names, groups = utils.group_by(cluster_table, len(cluster_table[0])-1)

# for each cluster, let's find out some stuff.
for i, group in enumerate(groups):
    values, counts = utils.get_frequencies(group, header.index("quality"))
    avg_att_vals = k_means.compute_average(group, len(group[0])-1)
    print("=" * 60)
    print("Cluster " + str(i+1) + ":")
    print("=" * 60)
    
    # get the most frequently occuring quality classification
    most_freq_qual_index = counts.index(max(counts))
    most_freq_quality = values[most_freq_qual_index]
    print("Most Frequent Classification: ", most_freq_quality)
    print()
    
    # print the average for each attribute
    print("Averages for Each Attribute ")
    for i, attribute in enumerate(avg_att_vals):
        print(header[i] + ": ", attribute)
    
    print("=" * 60)
    print()

### What can we take away? 

By observing the averages for each attribute in each cluster, we have discovered some attributes that vary. We noticed that the clusters with the most classifications of a quality 6 have some attributes that vary from clusters that are mostly a 5 rating. 

Classification 6 has:
- a slightly higher fixed acidity
- a lower residual sugar
- a lower free sulfur dioxide
- a lower total sulfur dioxide

We can inform the vineyard of this information and to carefully monitor these properties because it appears to have an affect on the quality.



# Classification Results


## Random Subsampling with K-Means Clustering
We used the centroids and clusters we created above to make predictions on random instances. We did this by taking random instances and finding what centroid it is closest to. Then, we find the majority class label of that cluster with majority voting. We tested our classifier by using random subsampling. For k times, we create a new training set and recalculated the centroids and clusters. Then we predict all the instances in the test set and compute the accuracy. You can see below that this classifier has a predictive accuracy of 83%. This is a high-performing classifier. I think that this is a high performer because most often the majority class of a cluster will be a 5 or 6. Since most of the instances in our dataset have a quality rating of 5 or 6 it will predict correctly for instances a good number of times. Below we run the k_means clustering with random subsampling and show its accuracy:

In [ ]:
#random subsampling 
random_avg_accuracy, random_stderr = k_means.random_subsampling(5, table, header.index("quality"), centroids, groups)
print("=" * 76)
print("Predictive Accuracy of K-Means Clustering with Random Subsampling")
print("=" * 76)
print("Accuracy = %0.2f, error rate = %0.2f" %(random_avg_accuracy, random_stderr))
print()

## Naive Bayes with Random Subsampling
The next classifier we used is Naive Bayes. This classifier uses prior and posterior probabilities and selects the classification with the highest probability to predict the label for each instance. It was designed to take in a list of indicies to include as posterior probabilities. We tested the classifier's predictive ability by using random subsampling. With this testing method, it has an accuracy of around 80%, fluctuating depending on what attributes you choose. 

Since we discerned that the fixed acidity, residual sugar, free sulfur dioxide, and total sulfur dioxide may result in a higher quality wine, we can test these out with our naive bayes classifier to see these attributes are more predictive of the quality of wine. The results show that the accuracy with these attributes are 81%. 

We also will test some other attributes, including residual sugar, density, and alcohol. We believe that college students will be more concerned with these attributes so we want to see what will be predicted by using these attributes. The results show that using Naive Bayes with these attributes results in an accuracy of 84%. 

While the second set of attributes had the highest accuracy, both are in the 80 percentile which is high. Like the classifier above it predicts 5 or 6 most of the time because the majority of the dataset is classified as 5 or 6 so the predictive accuracy is high. 

In [ ]:
import naive_bayes

# we noticed these attributes vary with K-means so lets see 
indices = [header.index("fixed acidity"),header.index('residual sugar'),header.index('free sulfur dioxide'), header.index('total sulfur dioxide')]
#['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
#'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

# Random SubSampling with Naive Bayes
random_avg_accuracy, random_stderr = naive_bayes.random_subsampling(5, table, header.index("quality"), indices)
print("=" * 76)
print("Predictive Accuracy of Naive Bayes with Guassian")
print("=" * 76)
print("Random Subsample (k=10, 2:1 Train/Test)")
print("Naive Bayes: accuracy = %0.2f, error rate = %0.2f" %(random_avg_accuracy, random_stderr))
print()


# college students care about these
indices = [header.index('residual sugar'),header.index('density'), header.index('alcohol')]


# Random SubSampling with Naive Bayes
random_avg_accuracy, random_stderr = naive_bayes.random_subsampling(5, table, header.index("quality"), indices)
print("=" * 76)
print("Predictive Accuracy of Naive Bayes with Guassian")
print("=" * 76)
print("Random Subsample (k=10, 2:1 Train/Test)")
print("Naive Bayes: accuracy = %0.2f, error rate = %0.2f" %(random_avg_accuracy, random_stderr))
print()

## KNN Using Random Subsampling
Using random subsampling, we can randomly select a train and test set and train/test our classifier on those sets. We repeat this process k times and take the average accuracy. 

In [10]:
utils.knn_random_subsampling(data, 5, [3, 7, 10], 11, 5)

instance: [6.9, 0.49, 0.1, 2.3, 0.074, 12.0, 30.0, 0.9959, 3.42, 0.58, 10.2, 6.0]
class: 5
actual: [6.0]
instance: [8.1, 0.785, 0.52, 2.0, 0.122, 37.0, 153.0, 0.9969, 3.21, 0.69, 9.3, 5.0]
class: 6
actual: [5.0]
instance: [7.8, 0.5, 0.3, 1.9, 0.075, 8.0, 22.0, 0.9959, 3.31, 0.56, 10.4, 6.0]
class: 6
actual: [6.0]
instance: [5.6, 0.31, 0.37, 1.4, 0.074, 12.0, 96.0, 0.9954, 3.32, 0.58, 9.2, 5.0]
class: 5
actual: [5.0]
instance: [7.8, 0.59, 0.33, 2.0, 0.074, 24.0, 120.0, 0.9968, 3.25, 0.54, 9.4, 5.0]
class: 5
actual: [5.0]


## Ensemble Classification Using Bootstrap Aggregation on KNN
With this classification approach, we first split our dataset into 1/3 testing and 2/3 remainder. Then, the remainder set is split into 2/3 training and 1/3 validation. For each of the k canditate classifiers, we train the classifier on our training set, and then determine it's accuracy over the validation set. We only use this classifier if it's accuracy is over a certain threshold. If it qualifies, it get's added to the ensemble.

Each classifier is trained and validated over a different set of random data collected using bagging. The bagging method involves taking our remaining data (size N), and randomly selecting N instances from this data (with replacement) to form our training set. Many instances will be selected more than once, and some not at all. Whatever isn't selected is put into the validation set. 

In [8]:
import ensemble_classification as es

def bootstrap_aggregation(table, num_class, attribute_indices, classification_index, min_acc, k):
    # runs classifiers over instances in the test set to produce predictions
    full_test = utils.generate_test_set(table)
    test_classifications = utils.get_classifications(full_test, classification_index)
    test = utils.normalize_instances(full_test, attribute_indices)
    # each approved training set and the accuracy of each said set
    training, accuracies = es.select_approved_training_sets(table, num_class, attribute_indices, classification_index, min_acc, k)
    correct = 0
    if(len(training) == 0):
        print("ensemble is empty")
    else:
        for index, sample in enumerate(test):
            for train_set in training:
                predictions = []
                # generate the prediction for this training set
                prediction = es.classifier_prediction(train_set, attribute_indices, classification_index, sample, k)
                predictions.append(prediction)
            # generate a final prediction from all classifiers using weighted majority voting
            final_prediction = utils.generate_weighted_majority_prediction(predictions, accuracies, test_classifications)
            full_sample = full_test[index]
            actual_classification = full_sample[classification_index]
            if (final_prediction == actual_classification):
                correct+= 1
        accuracy = correct/len(test)
        print("accuracy: "+str(accuracy))
        print("error rate: "+str(1-accuracy))
# use a subset of the dataset otherwise it takes a long time to run
data = table[:100]
bootstrap_aggregation(data, 20, [3, 7, 10], 11, .5, 5)


accuracy: 0.6060606060606061
error rate: 0.3939393939393939
